In [61]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Activation
from sklearn.preprocessing import MinMaxScaler
from keras.utils.data_utils import get_file
from keras.optimizers import RMSprop
import matplotlib.pyplot as plt
from tqdm import tqdm
import theano
import re

%matplotlib inline

## Download Text

In [63]:
path=get_file('t8.shakespeare.txt', origin="https://ocw.mit.edu/ans7870/6/6.006/s08/lecturenotes/files/t8.shakespeare.txt")
text=open(path)
text=text.readlines()[253:]
text=[t.strip() for t in text]
text="\n".join(text).lower()
text=re.sub(r'[^a-z\.\n\: ]',"",text)
text=re.sub(r'\n+',"\n",text)

print('corpus length:', len(text))

('corpus length:', 4798673)


In [64]:
chars=sorted(list(set(text)))
print('total chars:', len(chars))
char_indices=dict((c, i) for i, c in enumerate(chars))
indices_char=dict((i, c) for i, c in enumerate(chars))

('total chars:', 30)


## Generate Sentences and Predictions, ie.. the input output pairs. 
### For every set of 30 chars we predict 1 char.

In [65]:
maxlen=30
step=3
sentences=[]
next_chars=[]
y_shape=len(chars)
batch_size=100
for i in tqdm(range(0,(len(text)-maxlen),step)):
    sentences.append(text[i:i+maxlen])
    next_chars.append(text[i+maxlen])

trim=(len(sentences)/batch_size)*batch_size
sentences=sentences[:trim]
next_chars=next_chars[:trim]
print len(sentences)

100%|██████████| 1599548/1599548 [00:01<00:00, 1360765.00it/s]

1599500


# Vectorize 
### chars as one hot vectors

In [66]:
X=np.zeros((len(sentences), maxlen, y_shape), dtype=np.bool)
y=np.zeros((len(sentences), y_shape), dtype=np.bool)

for i, sentence in tqdm(enumerate(sentences)):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]]=1
        y[i, char_indices[next_chars[i]]]=1


1599500it [00:34, 45774.68it/s]


In [67]:
X.shape

(1599500, 30, 30)

## Stateful LSTMs

In [68]:
%%time
theano.config.compute_test_value="ignore"
model=Sequential()
model.add(LSTM(batch_size, input_shape=(maxlen,y_shape)))
model.add(Dropout(0.3))
model.add(Dense(y_shape))
model.add(Activation('softmax'))
optimizer=RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


CPU times: user 4.9 s, sys: 2.16 s, total: 7.06 s
Wall time: 4.69 s


In [69]:
def sample(probs, temp):
    #samples an index from prob array, depending on temp. This is used to get next starting index.
    probs=np.asarray(probs).astype('float64')
    log_like=np.log(probs)/temp #log likelihood/temp
    exp_preds=np.exp(log_like)
    preds=exp_preds / np.sum(exp_preds)
    probas=np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [70]:
model.fit(X, y, batch_size=batch_size, nb_epoch=1)


Epoch 1/1
1599500/1599500 [==============================] - 652s - loss: 1.8261   


In [71]:
gentext=''
first_idx=np.random.randint(0, len(text) - maxlen - 1)
first_line=text[firs]
gentext+=first_line
#print first_line, maxlen,first_line_index
for j in range(500):
            x=np.zeros((1, maxlen, y_shape))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]]=1.
                
            predic=model.predict(x, verbose=0)[0]
            temp= np.random.random()
            next_index=sample(predic, temp)
            next_char=indices_char[next_index]
            gentext+=next_char
            sentence=sentence[1:] + next_char
print gentext

The weather is pleasant todayrible the lord
he wistt perperint him me.
and the this young her sonsers and falls it grant and and the catraw of the now shall will shall be wirt this to the brother
and know and the  such the and king be the be the shes of the can to come can that these the now the ent to you princes
and i had the grace the cound the very in a can i ave i this that hath his pandon and have hose the thend of the so me i have the come whan the hand the ham the macks  i have on thou had so the benother the flace 
